<a href="https://colab.research.google.com/github/daddysin777/PyTorchDeepLearning/blob/main/Modular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import requests
import zipfile
from pathlib import Path

# Path to data folder
data_path = Path('data/')
image_path = data_path / 'pizza_steak_sushi'

# Create image folder if it doesn't exist
if image_path.is_dir():
  print(f"{image_path} directory exists")
else:
  print(f"{image_path} directory does not exist")
  image_path.mkdir(parents=True, exist_ok=True)

# Download data
with open(data_path / 'pizza_steak_sushi.zip', 'wb') as f:
  request = requests.get('https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip')
  print("Downloading...")
  f.write(request.content)

# Unzip data
with zipfile.ZipFile(data_path / 'pizza_steak_sushi.zip', 'r') as zip_ref:
  print()
  zip_ref.extractall(image_path)

# Remove zip file
os.remove(data_path / "pizza_steak_sushi.zip")
print("Done")

data/pizza_steak_sushi directory does not exist
Downloading...

Done


In [4]:
!mkdir going_modular

In [5]:
%%writefile going_modular/data_setup.py
"""
Creates dataloaders
"""
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(train_dir: str,
                       test_dir: str,
                       transform: transforms.Compose,
                       batch_size: int,
                       num_workers: int = NUM_WORKERS):
  """Creates training and testing DataLoaders.

  Takes in a training directory and testing directory path and turns
  them into PyTorch Datasets and then into PyTorch DataLoaders.

  Args:
    train_dir: Path to training directory.
    test_dir: Path to testing directory.
    transform: torchvision transforms to perform on training and testing data.
    batch_size: Number of samples per batch in each of the DataLoaders.
    num_workers: An integer for number of workers per DataLoader.

  Returns:
    A tuple of (train_dataloader, test_dataloader, class_names).
    Where class_names is a list of the target classes.
    Example usage:
      train_dataloader, test_dataloader, class_names = \
        = create_dataloaders(train_dir=path/to/train_dir,
                             test_dir=path/to/test_dir,
                             transform=some_transform,
                             batch_size=32,
                             num_workers=4)
  """
  train_data = datasets.ImageFolder(train_dir, transform=transform)
  test_data = datasets.ImageFolder(test_dir, transform=transform)

  class_names = train_data.classes

  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )

  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      pin_memory=True,
  )

  return train_dataloader, test_dataloader, class_names

Writing going_modular/data_setup.py


In [6]:
from going_modular import data_setup
from torchvision import transforms

data_transform = transforms.Compose([
    transforms.Resize(size=(64, 64)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor()
])

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=image_path / "train",
    test_dir=image_path / "test",
    transform=data_transform,
    batch_size=32,
)

In [7]:
%%writefile going_modular/model_builder2.py

"""
Creates TinyVGG model
"""

import torch
from torch import nn

class TinyVGG(nn.Module):
  """
  Args:
    input_shape: Integer indicating number of input channels
    hidden_units: Integer indicating number of hidden units
    output_shape: Integer indicating number of output classes
  """
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
    super().__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2)
    )

    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2)
    )

    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*13*13,
                  out_features=output_shape)
    )

  def forward(self, x: torch.Tensor):
    x = self.conv_block_1(x)
    x = self.conv_block_2(x)
    x = self.classifier(x)
    return x

Writing going_modular/model_builder2.py


In [8]:
import torch
from going_modular import model_builder2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model_builder2.TinyVGG(input_shape=3, hidden_units=10, output_shape=len(class_names)).to(device)

In [9]:
device

'cuda'

In [ ]:
model

TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1690, out_features=3, bias=True)
  )
)

In [10]:
%%writefile going_modular/engine.py
"""
Training and testing
"""

import torch

from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:

  """
  Perfomrms 1 training step

  Args:
    model: Pytorch model
    dataloader: Pytorch dataloader
    loss_fn: Pytorch loss function
    optimizer: Pytorch optimizer
    deivce: Target device to compute on(cuda or cpu)

  Returns:
    A tuple: (training loss, training accuracy)
  """

  model.train()
  train_loss, train_acc = 0, 0

  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += (y_pred_class == y).sum().item() / len(y_pred)

  train_loss /= len(dataloader)
  train_acc /= len(dataloader)
  return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> tuple[float, float]:

  """ Runs 1 test step of the model

  Args:
    model: Pytorch model
    dataloader: Pytorch dataloader
    loss_fn: Pytorch loss function
    device: Target device to compute on(cuda or cpu)

  Returns:
    A tuple: (test loss, test accuracy)
  """

  model.eval()
  test_loss, test_acc = 0, 0

  with torch.inference_mode():
    for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)

      y_pred = model(X)
      loss = loss_fn(y_pred, y)
      test_loss += loss.item()

      y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
      test_acc += (y_pred_class == y).sum().item() / len(y_pred)

  test_loss /= len(dataloader)
  test_acc /= len(dataloader)
  return test_loss, test_acc


def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          loss_fn: torch.nn.Module,
          optimizer: torch.optim.Optimizer,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
  """Trains and tests model

  Passes target pytorch model through train_step() and test_step() for a number of epochs
  Records the metrics in a dictionary

  Args:
    model: Pytorch model
    train_dataloader: Pytorch dataloader for training data
    test_dataloader: Pytorch dataloader for testing data
    loss_fn: Pytorch loss function
    optimizer: Pytorch optimizer
    epochs: Number of training epochs
    device: Target device to compute on(cuda or cpu)

  Returns:
    A dictionary of training and test losses and accuracies.

    Format:
      {
        "train_loss": [...],
        "train_acc": [...],
        "test_loss": [...],
        "test_acc": [...]
      }
  """
  history = {"train_loss": [],
             "train_acc": [],
             "test_loss": [],
             "test_acc": []}

  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model=model,
                                       dataloader=train_dataloader,
                                       loss_fn=loss_fn,
                                       optimizer=optimizer,
                                       device=device)
    test_loss, test_acc = test_step(model=model,
                                    dataloader=test_dataloader,
                                    loss_fn=loss_fn,
                                    device=device)

    print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
      )

    history["train_loss"].append(train_loss)
    history["train_acc"].append(train_acc)
    history["test_loss"].append(test_loss)
    history["test_acc"].append(test_acc)

  return history

Writing going_modular/engine.py


In [11]:
model.eval()
with torch.inference_mode():
  for (X, y) in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Shape of X [N, C, H, W]: torch.Size([32, 3, 64, 64])
Shape of y: torch.Size([32]) torch.int64


In [12]:
test_dataloader

In [13]:
from going_modular import engine

history = engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=torch.nn.CrossEntropyLoss(),
             optimizer=torch.optim.SGD(params=model.parameters(),
                                       lr=0.1),
             epochs=5,
             device=device)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.1027 | train_acc: 0.3828 | test_loss: 1.1736 | test_acc: 0.1979
Epoch: 2 | train_loss: 1.1069 | train_acc: 0.2930 | test_loss: 1.1169 | test_acc: 0.2604
Epoch: 3 | train_loss: 1.0972 | train_acc: 0.3047 | test_loss: 1.1379 | test_acc: 0.1979
Epoch: 4 | train_loss: 1.1006 | train_acc: 0.2930 | test_loss: 1.1366 | test_acc: 0.2604
Epoch: 5 | train_loss: 1.0963 | train_acc: 0.3047 | test_loss: 1.1592 | test_acc: 0.1979


In [22]:
%%writefile going_modular/utils.py
"""
Contains helper functions for training and saving
"""

import torch
from pathlib import Path

def save_model(model: torch.nn.Module, target_dir: Path, model_name: str):
  """Saves a PyTorch model to a target directory.

  Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.

  Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
  """
  # Create target directory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True, exist_ok=True)

  # Create model save path
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path / model_name

  # Save the model state_dict()
  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)

Writing going_modular/utils.py


In [23]:
from going_modular import utils

utils.save_model(model=model,
                 target_dir='models',
                 model_name='test')

TypeError: save_model() got an unexpected keyword argument 'model_name'

In [28]:
%%writefile going_modular/train.py
"""
Trains a pytorch model
"""

import os
import torch
import data_setup, engine, model_builder2, utils

from torchvision import transforms

# Constants
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

# Training and test directories
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

# Target device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Transforms
data_transform = transforms.Compose([
  transforms.Resize((64, 64)),
  transforms.ToTensor()
])

# Create dataloaders
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
      train_dir=train_dir,
      test_dir=test_dir,
      transform=data_transform,
      batch_size=BATCH_SIZE)

# Create model
model = model_builder2.TinyVGG(input_shape=3,
                               hidden_units=HIDDEN_UNITS,
                               output_shape=len(class_names)).to(device)

# Set loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(),
                            lr=LEARNING_RATE)

# Start training
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

# Save the model
utils.save_model(model=model,
                 target_dir='models',
                 model_name='model_1')

Overwriting going_modular/train.py


In [31]:
!python going_modular/train.py

  0% 0/5 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch: 1 | train_loss: 1.0991 | train_acc: 0.3047 | test_loss: 1.0986 | test_acc: 0.2604
 20% 1/5 [00:00<00:03,  1.15it/s]Epoch: 2 | train_loss: 1.0992 | train_acc: 0.3047 | test_loss: 1.0982 | test_acc: 0.2604
 40% 2/5 [00:01<00:02,  1.46it/s]Epoch: 3 | train_loss: 1.0986 | train_acc: 0.3047 | test_loss: 1.0977 | test_acc: 0.2604
 60% 3/5 [00:01<00:01,  1.64it/s]Epoch: 4 | train_loss: 1.1014 | train_acc: 0.3047 | test_loss: 1.0981 | test_acc: 0.2604
 80% 4/5 [00:02<00:00,  1.75it/s]Epoch: 5 | train_loss: 1.0952 | train_acc: 0.4258 | test_loss: 1.0984 | test_acc: 0.2604
100% 5/5 [00:02<00:00,  1.69it/s]
Traceback (most recent call last):
  File "/content/going_modular/train.py", line 57, in <module>
    utils.save_model(model=model,
